In [0]:
import os
import tensorflow as tf
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import cv2
from random import shuffle
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, AveragePooling2D, Input, Flatten, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.applications.xception import *
from keras.preprocessing import image
from keras import backend as K
from keras.models import Model, Sequential
import pickle 
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
num_classes=4
img_size=299
height=299
width=299
batch_size=16
epochs=15
scoring_set= "drive/My Drive/private_scoring"

In [0]:
dataset= "drive/My Drive/train/"
dir=os.listdir(dataset)
# test_dir= "D:/CNN/Datasets/test"

In [0]:
subtract_pixel_mean= False
data_augmentation= False

In [0]:
def create_train_data():
    total_train_images=0
    training_data= []
    label=0
    for i in dir:       
        class_dir= os.path.join(dataset, i) #return a list of string where string is the name of the image             
        images_list= os.listdir(class_dir)  
        print("class\t", i, len(images_list))  
        for image in tqdm(images_list):
            img_path= os.path.join(class_dir, image)            
            img_arr= cv2.imread(img_path, 1) # 0 for grayscale image >> img_arr is array of pixels
            new_img= cv2.resize(img_arr, (height, width))
            training_data.append([np.array(new_img),np.array(label)])
        print(label)
        label=label+1        
    shuffle(training_data)    
    return training_data

In [0]:
# None- 380
# car- 394
# bike- 339
# person- 286

training_data = create_train_data()
training_data= np.array(training_data)



  6%|▌         | 22/380 [00:00<00:01, 217.72it/s]

class	 none 380


  0%|          | 0/394 [00:00<?, ?it/s]

0
class	 cars 394


  0%|          | 0/339 [00:00<?, ?it/s]

1
class	 bike 339


  0%|          | 0/286 [00:00<?, ?it/s]

2
class	 person 286


100%|██████████| 286/286 [05:44<00:00,  1.20s/it]

3


In [0]:
#split into train and validation after shuffling of data
x_train= np.array([i[0] for i in training_data])
y_train= np.array([i[1] for i in training_data])


#split train data into train and val set
# x_train, x_val, y_train, y_val= train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print("shape after splitting train, val and test sets\n")
print(x_train.shape)
print(y_train.shape)
# print(x_val.shape)
# print(y_val.shape)

shape after splitting train, val and test sets

(1399, 299, 299, 3)
(1399,)


In [0]:
#one hot encoding
y_train= to_categorical(y_train, num_classes) 
# y_val= to_categorical(y_val, num_classes) 

print("after hot encoding", y_train.shape)
    
#reshape data
x_train = x_train.reshape(x_train.shape[0], img_size, img_size, 3)

# x_val = x_val.reshape(x_val.shape[0], img_size, img_size, 3)
# x_test = x_test.reshape(x_test.shape[0], img_size, img_size, 3)

print("shape after reshaping train, val and test sets\n")
print("train", x_train.shape)
# print("val", x_val.shape)
# print("test", x_test.shape)

#If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_val -= x_train_mean
    x_test -= x_train_mean

after hot encoding (1399, 4)
shape after reshaping train, val and test sets

train (1399, 299, 299, 3)


In [0]:
# Early stopping will stop the model from training before the number of epochs is reached if the model stops improving. We will set our
# early stopping monitor to 3. This means that after 3 epochs in a row in which the model doesn’t improve, training will stop. Sometimes, 
# the validation loss can stop improving then improve in the next epoch, but after 3 epochs in which the validation loss doesn’t improve, 
# it usually won’t improve again.

from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=10)

In [0]:
base_model = Xception(weights='imagenet', input_shape=(img_size, img_size, 3), include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

W0709 02:45:35.374341 140663823062912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0709 02:45:35.417170 140663823062912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0709 02:45:35.424211 140663823062912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0709 02:45:35.458673 140663823062912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0709 02:45:35.459583 1406638230

83689472/83683744 [==============================] - 6s 0us/step


W0709 02:45:50.966895 140663823062912 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(optimizer='Adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

W0709 02:45:51.036337 140663823062912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:



datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.5,
        zoom_range=(0.9, 1.1),
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='constant',
        cval=0
)


# get batch iterator
train_iterator = datagen.flow(x_train, y_train,batch_size=batch_size)


In [0]:
H = model.fit_generator(train_iterator, 
                       
                        shuffle=True,
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs, 
                        verbose=1, 
                        callbacks=[early_stopping_monitor])

Epoch 1/15
87/87 [==============================] - 58s 663ms/step - loss: 0.4053 - acc: 0.8793
Epoch 2/15


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


87/87 [==============================] - 58s 663ms/step - loss: 0.3471 - acc: 0.9021
Epoch 3/15
87/87 [==============================] - 57s 660ms/step - loss: 0.2562 - acc: 0.9278
Epoch 4/15
87/87 [==============================] - 57s 658ms/step - loss: 0.1812 - acc: 0.9423
Epoch 5/15
87/87 [==============================] - 57s 659ms/step - loss: 0.1241 - acc: 0.9612
Epoch 6/15
87/87 [==============================] - 57s 659ms/step - loss: 0.1612 - acc: 0.9583
Epoch 7/15
87/87 [==============================] - 57s 659ms/step - loss: 0.1153 - acc: 0.9670
Epoch 8/15
87/87 [==============================] - 57s 660ms/step - loss: 0.1090 - acc: 0.9655
Epoch 9/15
87/87 [==============================] - 57s 660ms/step - loss: 0.1008 - acc: 0.9727
Epoch 10/15
87/87 [==============================] - 57s 659ms/step - loss: 0.0884 - acc: 0.9806
Epoch 11/15
87/87 [==============================] - 57s 659ms/step - loss: 0.0672 - acc: 0.9835
Epoch 12/15
87/87 [==============================

In [0]:
model.save_weights('drive/My Drive/hack_training.h5')
# An H5 file is a data file saved in the Hierarchical Data Format (HDF). It contains multidimensional arrays of scientific data.

In [0]:
model.save("drive/My Drive/hack_training.model")